In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import Image

In [ ]:
display(Image(url="Data/tech_challenge_1.png", width=600, height=600), Image(url="Data/tech_challenge_2.png", width=600, height=600))

# Importar dados de exportação de vinhos

In [ ]:
dataset_exp = pd.read_csv('Data/ExpVinho.csv', sep=';')
dataset_exp.head()

# Importar dados de importação de vinhos

In [ ]:
dataset_imp = pd.read_csv('Data/ImpVinhos.csv', sep=';')
dataset_imp.head()

In [ ]:
dataset_exp.drop("Id", axis=1, inplace=True)
dataset_exp.info()

In [ ]:
dataset_imp.drop("Id", axis=1, inplace=True)
dataset_imp.info()

In [ ]:
dataset_exp.rename(columns={"País": "pais"}, inplace=True)
dataset_exp.head()

In [ ]:
dataset_imp.rename(columns={"País": "pais"}, inplace=True)
dataset_imp.head()

In [ ]:
colunas_exp = [coluna for coluna in dataset_exp.columns if '.1' in coluna]
colunas_exp.insert(0, 'pais')
dataset_exp_valor = dataset_exp[colunas_exp]
dataset_exp_valor.columns = dataset_exp_valor.columns.str.replace('.1', '')
dataset_exp_valor.info()

In [ ]:
colunas_imp = [coluna for coluna in dataset_exp.columns if '.1' in coluna]
colunas_imp.insert(0, 'pais')
dataset_imp_valor = dataset_imp[colunas_imp]
dataset_imp_valor.columns = dataset_imp_valor.columns.str.replace('.1', '')
dataset_imp_valor.info()

In [ ]:
colunas_exp.remove('pais')
dataset_exp_quantidade = dataset_exp.drop(colunas_exp, axis=1)
dataset_exp_quantidade.info()

In [ ]:
colunas_imp.remove('pais')
dataset_imp_quantidade = dataset_imp.drop(colunas_imp, axis=1)
dataset_imp_quantidade.info()

In [ ]:
dataset_exp_quantidade.head(3)

In [ ]:
dataset_exp_valor.head(3)

In [ ]:
dataset_imp_quantidade.head(3)

In [ ]:
dataset_imp_valor.head(3)

In [ ]:
dataset_exp_quantidade_unpivot = dataset_exp_quantidade.reset_index().melt(id_vars=["pais"], value_vars=dataset_exp_quantidade.columns)
dataset_exp_quantidade_unpivot.columns = ["Pais_destino", "ano", "Quantidade"]
dataset_exp_quantidade_unpivot.insert(0, 'Pais_origem', 'Brasil')
dataset_exp_quantidade_unpivot.ano = dataset_exp_quantidade_unpivot.ano.astype("datetime64[ns]").dt.year
dataset_exp_quantidade_unpivot

In [ ]:
dataset_imp_quantidade_unpivot = dataset_imp_quantidade.reset_index().melt(id_vars=["pais"], value_vars=dataset_imp_quantidade.columns)
dataset_imp_quantidade_unpivot.columns = ["Pais_destino", "ano", "Quantidade"]
dataset_imp_quantidade_unpivot.insert(0, 'Pais_origem', 'Brasil')
dataset_imp_quantidade_unpivot.ano = dataset_imp_quantidade_unpivot.ano.astype("datetime64[ns]").dt.year
dataset_imp_quantidade_unpivot

In [ ]:
dataset_exp_valor_unpivot = dataset_exp_valor.reset_index().melt(id_vars=["pais"], value_vars=dataset_exp_valor.columns)
dataset_exp_valor_unpivot.columns = ["Pais_destino", "ano", "Valor"]
dataset_exp_valor_unpivot.insert(0, 'Pais_origem', 'Brasil')
dataset_exp_valor_unpivot.ano = dataset_exp_valor_unpivot.ano.astype("datetime64[ns]").dt.year
dataset_exp_valor_unpivot

In [ ]:
dataset_imp_valor_unpivot = dataset_imp_valor.reset_index().melt(id_vars=["pais"], value_vars=dataset_imp_valor.columns)
dataset_imp_valor_unpivot.columns = ["Pais_destino", "ano", "Valor"]
dataset_imp_valor_unpivot.insert(0, 'Pais_origem', 'Brasil')
dataset_imp_valor_unpivot.ano = dataset_imp_valor_unpivot.ano.astype("datetime64[ns]").dt.year
dataset_imp_valor_unpivot

In [ ]:
dataset_exp_unpivot = pd.merge(
    dataset_exp_quantidade_unpivot,
    dataset_exp_valor_unpivot,
    left_on=['Pais_origem', 'Pais_destino', 'ano'],
    right_on=['Pais_origem', 'Pais_destino', 'ano'])
dataset_exp_unpivot

In [ ]:
dataset_imp_unpivot = pd.merge(
    dataset_imp_quantidade_unpivot,
    dataset_imp_valor_unpivot,
    left_on=['Pais_origem', 'Pais_destino', 'ano'],
    right_on=['Pais_origem', 'Pais_destino', 'ano'])
dataset_imp_unpivot = dataset_imp_unpivot.drop(dataset_imp_unpivot.index[-3:])
dataset_imp_unpivot

In [ ]:
dataset_exp_periodo = dataset_exp_unpivot.query("ano >= 2007")
dataset_exp_periodo.ano.unique()

In [ ]:
dataset_imp_periodo = dataset_imp_unpivot.query("ano >= 2007")
dataset_imp_periodo.ano.unique()

In [ ]:
dataset_exp_periodo.head(5)

In [ ]:
dataset_imp_periodo.head(5)

In [ ]:
total_exp_ano = dataset_exp_periodo.groupby('ano').sum()
total_exp_ano = total_exp_ano.iloc[:,2:4]
total_exp_ano.head()

In [ ]:
total_imp_ano = dataset_imp_periodo.groupby('ano').sum()
total_imp_ano = total_imp_ano.iloc[:,2:4]
total_imp_ano.head()

# Dados para gráfico de porcentagem anual

In [ ]:
#Dataset com 124 países e 15 anos
dataset_exp_periodo

In [ ]:
df_temporal=dataset_exp_periodo.iloc[:,[0,1,2,4]]
df_temporal

In [ ]:
df_temporal=df_temporal.pivot(index='ano', columns='Pais_destino', values='Valor')


In [ ]:
df_temporal=df_temporal/1000_000

In [ ]:
df_temporal

In [ ]:
paises_separados = ['Rússia', 'Paraguai','Estados Unidos','Reino Unido', 'China']
df_anos=df_temporal[['Rússia', 'Paraguai','Estados Unidos','Reino Unido', 'China']]
df_anos['Outros'] = df_temporal.drop(paises_separados, axis=1).sum(axis=1)
df_anos

In [ ]:
df_anos=df_anos[['Outros','China','Reino Unido','Estados Unidos','Paraguai','Rússia']]
df_anos

In [ ]:
def plot_per_anual(data):  
    
    cores = {
        'Rússia': 'rgb(255, 0, 0)',     # Vermelho
        'Paraguai': 'rgb(200, 162, 200)',   # Lilás
        'Estados Unidos': 'rgb(0, 0, 255)',     # Azul
        'Reino Unido':'rgb(128, 0, 128)',   # Roxo
        'China': 'rgb(255, 255, 0)',   # Amarelo
        'Outros': 'rgb(128, 128, 128)'      # Cinza
    }
    data_percent=data.mul(100).div(data.sum(axis=1),axis=0)
    fig = go.Figure()

    for coluna in data.columns:
        fig.add_trace(go.Bar(
            x=data.index,
            y=data[coluna],
            name=coluna,
            marker=dict(color=cores[coluna]),
            text=data_percent[coluna].apply(lambda x: f'{x:.1f}%'),  
            textposition='auto' 
        ))

    fig.update_layout(
        title='Percentual de exportação anual dos países com maior influência',
        barmode='stack',
        legend=dict(x=1, y=1, orientation='v'),
        xaxis=dict(tickmode='linear'),
        height=800,
        yaxis_title='Valor (em milhões de U$)'
    )
    fig.show()


In [ ]:
plot_per_anual(df_anos)

# Dados para Pareto de 2016 a 2021

In [ ]:
total_exp_5 =dataset_exp_periodo.query("ano>=2016")
total_exp_5 = total_exp_5.groupby('ano').sum()
total_exp_5 = total_exp_5.iloc[:,2:4]
total_exp_5.head()

In [ ]:
total_exp_5 = dataset_exp_periodo.query("ano>=2016").groupby(['Pais_destino','Pais_origem']).sum()
total_exp_5=total_exp_5.drop(columns=['ano','Quantidade'],axis=1)
total_exp_5.head()

In [ ]:
exp_pareto_5 = total_exp_5.sort_values('Valor', ascending=False)
exp_pareto_5['Porcentagem_acumulada_valor'] = (exp_pareto_5['Valor'].cumsum() / exp_pareto_5['Valor'].sum()) * 100
exp_pareto_5

# Criar tabela com itens de "a" até "d"

In [ ]:
total_exp_pais = dataset_exp_periodo.groupby(['Pais_destino', 'Pais_origem']).sum()
total_exp_pais = total_exp_pais.iloc[:,:4].sort_values(by=['Quantidade', 'Valor'], ascending=False)
total_exp_pais.drop(columns=['ano'], inplace=True)
total_exp_pais.reset_index(inplace=True)
total_exp_pais = total_exp_pais[(total_exp_pais.Quantidade != 0) & (total_exp_pais.Valor != 0)]
total_exp_pais.head()

In [ ]:
total_imp_pais = dataset_imp_periodo.groupby(['Pais_destino', 'Pais_origem']).sum()
total_imp_pais = total_imp_pais.iloc[:,:4].sort_values(by=['Quantidade', 'Valor'], ascending=False)
total_imp_pais.drop(columns=['ano'], inplace=True)
total_imp_pais.reset_index(inplace=True)
total_imp_pais = total_imp_pais.drop(total_imp_pais.index[-3:])
total_imp_pais = total_imp_pais[(total_imp_pais.Quantidade != 0) & (total_imp_pais.Valor != 0)]
total_imp_pais.head()

In [ ]:
total_exp = dataset_exp_periodo.groupby(['Pais_origem', 'Pais_destino', 'ano']).sum()
total_exp.head()

In [ ]:
total_imp = dataset_imp_periodo.groupby(['Pais_origem', 'Pais_destino', 'ano']).sum()
total_imp.head()

In [ ]:
dataset_exp_periodo[['Quantidade', 'Valor']].describe().round()

In [ ]:
dataset_imp_periodo[['Quantidade', 'Valor']].describe().round()

In [ ]:
total_exp_pais.replace('Alemanha, República Democrática', 'Alemanha', inplace=True)
total_exp_pais.replace('Taiwan (FORMOSA)', 'Taiwan', inplace=True)
total_exp_pais.replace('Tcheca, República', 'República Tcheca', inplace=True)
total_exp_pais.replace('Marshall, Ilhas', 'Ilhas Marshall', inplace=True)
total_exp_pais.replace('Coreia, Republica Sul', 'Coreia do Sul', inplace=True)
total_exp_pais.replace('Eslovaca, Republica', 'Eslováquia', inplace=True)

In [ ]:
total_imp_pais.replace('Alemanha, República Democrática', 'Alemanha', inplace=True)
total_imp_pais.replace('Taiwan (FORMOSA)', 'Taiwan', inplace=True)
total_imp_pais.replace('Tcheca, República', 'República Tcheca', inplace=True)
total_imp_pais.replace('Geórgia do Sul e Sandwich do Sul, Ilhas', 'Ilhas Geórgia do Sul e Sandwich do Sul,', inplace=True)
total_imp_pais.replace('Paises Baixos (Holanda)', 'Países Baixos', inplace=True)
total_imp_pais.replace('Tcheca, República', 'República Tcheca', inplace=True)

### Salvar .CSV para uso no Streamlit

In [ ]:
total_exp_pais.to_csv('ExpVinhoPeriodo.csv', sep=',', index=False)
total_imp_pais.to_csv('ImpVinhosPeriodo.csv', sep=',', index=False)

In [ ]:
total_exp_pareto = total_exp_pais.sort_values('Valor', ascending=False)
total_exp_pareto['Porcentagem_acumulada_valor'] = (total_exp_pareto['Valor'].cumsum() / total_exp_pareto['Valor'].sum()) * 100
totalq_exp_pareto = total_exp_pais.sort_values('Quantidade', ascending=False)
totalq_exp_pareto['Porcentagem_acumulada_quantidade'] = (totalq_exp_pareto['Quantidade'].cumsum() / totalq_exp_pareto['Quantidade'].sum()) * 100
total_exp_pareto

In [ ]:
totalq_exp_pareto.head(10)

In [ ]:
total_imp_pareto = total_imp_pais.sort_values('Valor', ascending=False)
total_imp_pareto['Porcentagem_acumulada_valor'] = (total_imp_pareto['Valor'].cumsum() / total_imp_pareto['Valor'].sum()) * 100
total_imp_pareto['Porcentagem_acumulada_quantidade'] = (total_imp_pareto['Quantidade'].cumsum() / total_imp_pareto['Quantidade'].sum()) * 100
total_imp_pareto

### Salvar .CSV para uso no Streamlit

In [ ]:
total_exp_pareto.to_csv('ExpPareto.csv', sep=',', index=False)
totalq_exp_pareto.to_csv('QExpPareto.csv', sep=',', index=False)
total_imp_pareto.to_csv('ImpPareto.csv', sep=',', index=False)
exp_pareto_5.to_csv('ExpPareto5.csv', sep=',')
df_anos.to_csv('PorcentAnual.csv', sep=',')

In [ ]:
total_exp_pareto

- fornecer o montante de venda de exportação dos últimos 15 anos
- separando a analise por país
- e trazendo quais as prospercções futuras e possíveis ações para uma melhoria nas exportações
- Construa gráfico atraentes e que passem a ideia central para que os acionistas e invetidores possam seguir em frente com suas ações.

In [ ]:
total_exp_pareto_81 = total_exp_pareto.query("Porcentagem_acumulada_valor < 81")
total_exp_pareto_81

In [ ]:
totalq_exp_pareto_92 = totalq_exp_pareto.query(" Porcentagem_acumulada_quantidade < 92")

In [ ]:
total_imp_pareto = total_imp_pareto.query("Porcentagem_acumulada_valor < 90 or Porcentagem_acumulada_quantidade < 90")
total_imp_pareto

In [ ]:
def plot_pareto(data, bar_x, bar_y, scatter_x, scatter_y, title, title_x, title_y, mean):
    pareto = px.bar(
        data,
        x=bar_x,
        y=bar_y,
        color=bar_x,
        title=bar_x,
        color_discrete_sequence=px.colors.qualitative.T10,
        template='plotly_white',
        text_auto='.2s'
    )
    pareto.update_layout(
        width=1300,
        height=600
    )
    pareto.add_scatter(
        x=data[scatter_x],
        y=data[scatter_y], 
        mode='lines+text',
        name='Porcentagem Acumulada',
        text=[f'{x:.2f}' for x in data[scatter_y]],
        textposition='top center',
        yaxis='y2',
        line={'color': 'red'}
    )
    pareto.update_layout(
        title=title,
        xaxis=dict(title=title_x),
        yaxis=dict(title=title_y),
        yaxis2=dict(title='Porcentagem Acumulada', overlaying='y', side='right', showgrid=False, range=[0, 100]),
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )
    pareto.add_shape(
        type='line',
        x0=0,
        x1=1,
        y0=80,
        y1=80,
        line=dict(color='black', width=1),
        xref='paper',
        yref='y2'
    )
    pareto.add_shape(
        type='line',
        x0=0,
        x1=7,
        y0=mean,
        y1=mean,
        line=dict(color='blue', width=1)
    )
    pareto.add_annotation(
        x=4.5,
        y=mean,
        text= f"Média: {mean / 1000000:.2f}M",
        showarrow=True,
        arrowhead=1,
        ax=40,
        ay=-40,
        xref='x',
        font=dict(color='blue')
    )
    pareto.show()

## Gráfico de Pareto dados de Exportação

In [ ]:
plot_pareto(
    total_exp_pareto_81,
    "Pais_destino",
    "Valor",
    "Pais_destino",
    "Porcentagem_acumulada_valor",
    "Gráfico de Pareto - Valor Total de Vendas para países responsáveis por 80% da vendas no período",
    "País",
    "Valor (U$)",
    total_exp_pareto.Valor.mean()
)

**Paraguai, Rússia, Estados Unidos, Reino Unido, China, Países Baixos e Espanha são responsáveis por 80% do total vendido**

In [ ]:
plot_pareto(
    totalq_exp_pareto_92,
    "Pais_destino",
    "Quantidade",
    "Pais_destino",
    "Porcentagem_acumulada_quantidade",
    "Gráfico de Pareto - Quantidade Total de Vendas para países responsáveis por 90% das quantidades vendidas no período",
    "País",
    "Quantidade (L)",
    totalq_exp_pareto.Quantidade.mean()
)

**Paraguai, Rússia e EUA são responsáveis por 80% do total vendido em Kg/Litros**

In [ ]:
plot_pareto(
    exp_pareto_5.query("Porcentagem_acumulada_valor < 90").reset_index(),
    "Pais_destino",
    "Valor",
    "Pais_destino",
    "Porcentagem_acumulada_valor",
    "Gráfico de Pareto - Valor Total de Vendas parar países responsáveis por até 90% da vendas no período de 2016 a 2021",
    "País",
    "Valor (U$)",
    exp_pareto_5.Valor.mean()
)

In [ ]:
# plot_pareto(
#     exp_pareto_2014.query("Porcentagem_acumulada_valor < 85").reset_index(),
#     "Pais_destino",
#     "Valor",
#     "Pais_destino",
#     "Porcentagem_acumulada_valor",
#     "Gráfico de Pareto - Valor Total de Vendas para países responsáveis por até 85% da vendas em 2014",
#     "País",
#     "Valor (U$)",
#     exp_pareto_2014.Valor.mean()
# )

## Gráfico de Pareto dados de Importação

In [ ]:
plot_pareto(
    total_imp_pareto,
    "Pais_destino",
    "Valor",
    "Pais_destino",
    "Porcentagem_acumulada_valor",
    "Gráfico de Pareto - Valor Total de Importação para países responsáveis por 80% das importações no período",
    "País",
    "Valor (U$)",
    total_imp_pareto.Valor.mean()
)

**Chile, Argentina, Portugal e Itália são responsáveis por 80% do total importado**

In [ ]:
plot_pareto(
    total_imp_pareto,
    "Pais_destino",
    "Quantidade",
    "Pais_destino",
    "Porcentagem_acumulada_quantidade",
    "Gráfico de Pareto - Quantidade Total de Importação para países responsáveis por 80% das importações no período",
    "País",
    "Quantidade (L)",
    total_imp_pareto.Quantidade.mean()
)

**Chile, Argentina, Portugal e Itália são responsáveis por 80% da quantidade importada**

# Importar dados financeiros do Banco Mundial

## Dados do PIB (US$)

In [ ]:
dataset_pib = pd.read_excel('data/GPD.xlsx', thousands=',', decimal='.', skipfooter=50)
dataset_pib.drop('Unnamed: 4', axis=1, inplace=True)
dataset_pib.rename(columns={" ": "pais"}, inplace=True)
dataset_pib.head(1)

## Dados de Inflação (%)

In [ ]:
dataset_inflation = pd.read_excel('data/Inflation.xlsx', skipfooter=54)
dataset_inflation.drop('Series Name', axis=1, inplace=True)
dataset_inflation.drop('Series Code', axis=1, inplace=True)
dataset_inflation.drop('Country Code', axis=1, inplace=True)
dataset_inflation.rename(columns={"Country Name": "pais"}, inplace=True)
dataset_inflation.columns = dataset_inflation.columns.str.replace(r'\[YR\d{4}\]\s*', '', regex=True)
dataset_inflation.head(1)

## Dados de proporção do comércio internacional em relação ao PIB (%) 

In [ ]:
dataset_trade = pd.read_excel('data/Trade.xlsx', skipfooter=54)
dataset_trade.drop('Series Name', axis=1, inplace=True)
dataset_trade.drop('Series Code', axis=1, inplace=True)
dataset_trade.drop('Country Code', axis=1, inplace=True)
dataset_trade.rename(columns={"Country Name": "pais"}, inplace=True)
dataset_trade.columns = dataset_trade.columns.str.replace(r'\[YR\d{4}\]\s*', '', regex=True)
dataset_trade.head(2)

## Dados de população

In [ ]:
dataset_population = pd.read_excel('data/Population.xlsx', skipfooter=54)
dataset_population.drop('Series Name', axis=1, inplace=True)
dataset_population.drop('Series Code', axis=1, inplace=True)
dataset_population.drop('Country Code', axis=1, inplace=True)
dataset_population.rename(columns={"Country Name": "pais"}, inplace=True)
dataset_population.columns = dataset_population.columns.str.replace(r'\[YR\d{4}\]\s*', '', regex=True)
dataset_population.head(1)

## Dados de desemprego

In [ ]:
dataset_unemployment = pd.read_excel('data/Unemployment.xlsx', skipfooter=54)
dataset_unemployment.drop('Series Name', axis=1, inplace=True)
dataset_unemployment.drop('Series Code', axis=1, inplace=True)
dataset_unemployment.drop('Country Code', axis=1, inplace=True)
dataset_unemployment.rename(columns={"Country Name": "pais"}, inplace=True)
dataset_unemployment.columns = dataset_unemployment.columns.str.replace(r'\[YR\d{4}\]\s*', '', regex=True)
dataset_unemployment.head(1)

# Limpeza dos dados

In [ ]:
dataset_pib.set_index('pais',inplace=True)
dataset_pib

In [ ]:
dataset_inflation.set_index('pais',inplace=True)
dataset_inflation

In [ ]:
dataset_trade.set_index('pais',inplace=True)
dataset_trade

In [ ]:
dataset_population.set_index('pais',inplace=True)
dataset_population

In [ ]:
dataset_unemployment.set_index('pais',inplace=True)
dataset_unemployment

In [ ]:
dataset_pib.info()

In [ ]:
dataset_pib = dataset_pib.replace("..", "0").applymap(lambda x: float(x) if isinstance(x, str) else x)
dataset_pib.reset_index(inplace=True)
dataset_pib

In [ ]:
dataset_pib.info()

In [ ]:
dataset_inflation.info()

In [ ]:
dataset_inflation = dataset_inflation.replace("..", "0").applymap(lambda x: float(x) if isinstance(x, str) else x)
dataset_inflation.reset_index(inplace=True)
dataset_inflation

In [ ]:
dataset_inflation.info()

In [ ]:
dataset_trade.info()

In [ ]:
dataset_trade = dataset_trade.replace("..", "0").applymap(lambda x: float(x) if isinstance(x, str) else x)
dataset_trade.reset_index(inplace=True)
dataset_trade

In [ ]:
dataset_trade.info()

In [ ]:
dataset_population.info()

In [ ]:
dataset_population = dataset_population.applymap(lambda x: float(x) if isinstance(x, str) else x)
dataset_population.reset_index(inplace=True)
dataset_population

In [ ]:
dataset_population.info()

In [ ]:
dataset_unemployment.info()

In [ ]:
dataset_unemployment = dataset_unemployment.replace("..", "0").applymap(lambda x: float(x) if isinstance(x, str) else x)
dataset_unemployment.reset_index(inplace=True)
dataset_unemployment

In [ ]:
dataset_unemployment.info()

In [ ]:
dataset_exp_valor['pais'] = dataset_exp_valor['pais'].str.replace('Alemanha, República Democrática', 'Alemanha')
dataset_exp_valor['pais'] = dataset_exp_valor['pais'].str.replace('Taiwan (FORMOSA)', 'Taiwan')
paises_exportacao = dataset_exp_valor.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha' or pais == 'Alemanha' or pais == 'Japão' or pais == 'Haiti' or pais == 'Bélgica' or pais == 'Canadá' or pais == 'Suíça'")
periodo_exp = [str(ano) for ano in range(2007, 2022)]
colunas_exp = ['pais'] + periodo_exp
paises_exportacao = paises_exportacao.loc[:, colunas_exp]
paises_exportacao.head()

In [ ]:
# paises_importacao = dataset_imp_valor.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")
paises_importacao = dataset_imp_valor.query("pais == 'Chile' or pais == 'Argentina' or pais == 'Portugal' or pais == 'Itália' or pais == 'França' or pais == 'Espanha' or pais == 'Uruguai' or pais == 'Africa do Sul' or pais == 'Estados Unidos' or pais == 'Austrália' or pais == 'Nova Zelândia' or pais == 'Alemanha'")
periodo_imp = [str(ano) for ano in range(2007, 2022)]
colunas_imp = ['pais'] + periodo_imp
paises_importacao = paises_importacao.loc[:, colunas_imp]
paises_importacao

In [ ]:
paises_exportacao.dtypes

In [ ]:
paises_importacao.dtypes

In [ ]:
nomes = {
    'Afghanistan': 'Afeganistão',
    'Albania': 'Albânia',
    'Algeria': 'Argélia',
    'American Samoa': 'Samoa',
    'Andorra': 'Andorra',
    'Angola': 'Angola',
    'Antigua and Barbuda': 'Antígua e Barbuda',
    'Argentina': 'Argentina',
    'Armenia': 'Armênia',
    'Aruba': 'Aruba',
    'Australia': 'Austrália',
    'Austria': 'Áustria',
    'Azerbaijan': 'Azerbaijão',
    'Bahamas, The': 'Bahamas',
    'Bahrain': 'Bahrein',
    'Bangladesh': 'Bangladesh',
    'Barbados': 'Barbados',
    'Belarus': 'Bielorrússia',
    'Belgium': 'Bélgica',
    'Belize': 'Belize',
    'Benin': 'Benin',
    'Bermuda': 'Bermudas',
    'Bhutan': 'Butão',
    'Bolivia': 'Bolívia',
    'Bosnia and Herzegovina': 'Bósnia e Herzegovina',
    'Botswana': 'Botsuana',
    'Brazil': 'Brasil',
    'British Virgin Islands': 'Ilhas Virgens Britânicas',
    'Brunei Darussalam': 'Brunei',
    'Bulgaria': 'Bulgária',
    'Burkina Faso': 'Burkina Faso',
    'Burundi': 'Burundi',
    'Cabo Verde': 'Cabo Verde',
    'Cambodia': 'Camboja',
    'Cameroon': 'Camarões',
    'Canada': 'Canadá',
    'Cayman Islands': 'Ilhas Cayman',
    'Central African Republic': 'República Centro-Africana',
    'Chad': 'Chade',
    'Channel Islands': 'Ilhas do Canal',
    'Chile': 'Chile',
    'China': 'China',
    'Colombia': 'Colômbia',
    'Comoros': 'Comores',
    'Congo, Dem. Rep.': 'Dem Congo',
    'Congo, Rep.': 'Congo',
    'Costa Rica': 'Costa Rica',
    "Cote d'Ivoire": 'Costa do Marfim',
    'Croatia': 'Croácia',
    'Cuba': 'Cuba',
    'Curacao': 'Curaçao',
    'Cyprus': 'Chipre',
    'Czechia': 'Tchéquia',
    'Denmark': 'Dinamarca',
    'Djibouti': 'Djibuti',
    'Dominica': 'Dominica',
    'Dominican Republic': 'República Dominicana',
    'Ecuador': 'Equador',
    'Egypt, Arab Rep.': 'Egito',
    'El Salvador': 'El Salvador',
    'Equatorial Guinea': 'Guiné Equatorial',
    'Eritrea': 'Eritreia',
    'Estonia': 'Estônia',
    'Eswatini': 'Eswatini',
    'Ethiopia': 'Etiópia',
    'Faroe Islands': 'Ilhas Faroe',
    'Fiji': 'Fiji',
    'Finland': 'Finlândia',
    'France': 'França',
    'French Polynesia': 'Polinésia',
    'Gabon': 'Gabão',
    'Gambia, The': 'Gâmbia',
    'Georgia': 'Geórgia',
    'Germany': 'Alemanha',
    'Ghana': 'Gana',
    'Gibraltar': 'Gibraltar',
    'Greece': 'Grécia',
    'Greenland': 'Groenlândia',
    'Grenada': 'Granada',
    'Guam': 'Guam',
    'Guatemala': 'Guatemala',
    'Guinea': 'Guiné',
    'Guinea-Bissau': 'Guiné-Bissau',
    'Guyana': 'Guiana',
    'Haiti': 'Haiti',
    'Honduras': 'Honduras',
    'Hong Kong SAR, China': 'Hong Kong',
    'Hungary': 'Hungria',
    'Iceland': 'Islândia',
    'India': 'Índia',
    'Indonesia': 'Indonésia',
    'Iran, Islamic Rep.': 'Irã',
    'Iraq': 'Iraque',
    'Ireland': 'Irlanda',
    'Isle of Man': 'Ilha de Man',
    'Israel': 'Israel',
    'Italy': 'Itália',
    'Jamaica': 'Jamaica',
    'Japan': 'Japão',
    'Jordan': 'Jordânia',
    'Kazakhstan': 'Cazaquistão',
    'Kenya': 'Quênia',
    'Kiribati': 'Kiribati',
    "Korea, Dem. People's Rep.": 'Coreia do Norte',
    'Korea, Rep.': 'Coreia do Sul',
    'Kosovo': 'Kosovo',
    'Kuwait': 'Kuwait',
    'Kyrgyz Republic': 'Quirguistão',
    'Lao PDR': 'Laos',
    'Latvia': 'Letônia',
    'Lebanon': 'Líbano',
    'Lesotho': 'Lesoto',
    'Liberia': 'Libéria',
    'Libya': 'Líbia',
    'Liechtenstein': 'Liechtenstein',
    'Lithuania': 'Lituânia',
    'Luxembourg': 'Luxemburgo',
    'Macao SAR, China': 'Macau',
    'Madagascar': 'Madagascar',
    'Malawi': 'Malawi',
    'Malaysia': 'Malásia',
    'Maldives': 'Maldivas',
    'Mali': 'Mali',
    'Malta': 'Malta',
    'Marshall Islands': 'Ilhas Marshall',
    'Mauritania': 'Mauritânia',
    'Mauritius': 'Maurício',
    'Mexico': 'México',
    'Micronesia, Fed. Sts.': 'Micronésia',
    'Moldova': 'Moldávia',
    'Monaco': 'Mônaco',
    'Mongolia': 'Mongólia',
    'Montenegro': 'Montenegro',
    'Morocco': 'Marrocos',
    'Mozambique': 'Moçambique',
    'Myanmar': 'Mianmar',
    'Namibia': 'Namíbia',
    'Nauru': 'Nauru',
    'Nepal': 'Nepal',
    'Netherlands': 'Países Baixos',
    'New Caledonia': 'Nova Caledônia',
    'New Zealand': 'Nova Zelândia',
    'Nicaragua': 'Nicarágua',
    'Niger': 'Níger',
    'Nigeria': 'Nigéria',
    'North Macedonia': 'Macedônia do Norte',
    'Northern Mariana Islands': 'Ilhas Marianas do Norte',
    'Norway': 'Noruega',
    'Oman': 'Omã',
    'Pakistan': 'Paquistão',
    'Palau': 'Palau',
    'Panama': 'Panamá',
    'Papua New Guinea': 'Papua-Nova Guiné',
    'Paraguay': 'Paraguai',
    'Peru': 'Peru',
    'Philippines': 'Filipinas',
    'Poland': 'Polônia',
    'Portugal': 'Portugal',
    'Puerto Rico': 'Porto Rico',
    'Qatar': 'Catar',
    'Romania': 'Romênia',
    'Russian Federation': 'Rússia',
    'Rwanda': 'Ruanda',
    'Samoa': 'Samoa',
    'San Marino': 'San Marino',
    'Sao Tome and Principe': 'São Tomé e Príncipe',
    'Saudi Arabia': 'Arábia Saudita',
    'Senegal': 'Senegal',
    'Serbia': 'Sérvia',
    'Seychelles': 'Seychelles',
    'Sierra Leone': 'Serra Leoa',
    'Singapore': 'Singapura',
    'Sint Maarten (Dutch part)': 'São Martinho',
    'Slovak Republic': 'Eslováquia',
    'Slovenia': 'Eslovênia',
    'Solomon Islands': 'Ilhas Salomão',
    'Somalia': 'Somália',
    'South Africa': 'África do Sul',
    'South Sudan': 'Sudão do Sul',
    'Spain': 'Espanha',
    'Sri Lanka': 'Sri Lanka',
    'St. Kitts and Nevis': 'São Cristóvão e Névis',
    'St. Lucia': 'Santa Lúcia',
    'St. Martin (French part)': 'São Martinho',
    'St. Vincent and the Grenadines': 'São Vicente e Granadinas',
    'Sudan': 'Sudão',
    'Suriname': 'Suriname',
    'Sweden': 'Suécia',
    'Switzerland': 'Suíça',
    'Syrian Arab Republic': 'Síria',
    'Tajikistan': 'Tajiquistão',
    'Tanzania': 'Tanzânia',
    'Thailand': 'Tailândia',
    'Timor-Leste': 'Timor-Leste',
    'Togo': 'Togo',
    'Tonga': 'Tonga',
    'Trinidad and Tobago': 'Trinidad e Tobago',
    'Tunisia': 'Tunísia',
    'Turkiye': 'Turquia',
    'Turkmenistan': 'Turcomenistão',
    'Turks and Caicos Islands': 'Ilhas Turks e Caicos',
    'Tuvalu': 'Tuvalu',
    'Uganda': 'Uganda',
    'Ukraine': 'Ucrânia',
    'United Arab Emirates': 'Emirados Árabes Unidos',
    'United Kingdom': 'Reino Unido',
    'United States': 'Estados Unidos',
    'Uruguay': 'Uruguai',
    'Uzbekistan': 'Uzbequistão',
    'Vanuatu': 'Vanuatu',
    'Venezuela, RB': 'Venezuela',
    'Vietnam': 'Vietnã',
    'Virgin Islands (U.S.)': 'Ilhas Virgens',
    'West Bank and Gaza': 'Cisjordânia e Faixa de Gaza',
    'Yemen, Rep.': 'Iêmen',
    'Zambia': 'Zâmbia',
    'Zimbabwe': 'Zimbábue'
}

dataset_pib["pais"] = dataset_pib["pais"].map(nomes)
dataset_inflation["pais"] = dataset_inflation["pais"].map(nomes)
dataset_trade["pais"] = dataset_trade["pais"].map(nomes)
dataset_population["pais"] = dataset_population["pais"].map(nomes)
dataset_unemployment["pais"] = dataset_unemployment["pais"].map(nomes)

### Salvar .CSV para uso no Streamlit

In [ ]:
dataset_pib.to_csv('Pib.csv', sep=',', index=False)
dataset_inflation.to_csv('Inflation.csv', sep=',', index=False)
dataset_trade.to_csv('Trade.csv', sep=',', index=False)
dataset_population.to_csv('Population.csv', sep=',', index=False)
dataset_unemployment.to_csv('Unemployment.csv', sep=',', index=False)
paises_exportacao.to_csv('Exportacao.csv', sep=',', index=False)
paises_importacao.to_csv('Importacao.csv', sep=',', index=False)

In [ ]:
paises_pib = dataset_pib.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")
paises_inflation = dataset_inflation.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")
paises_trade = dataset_trade.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")
paises_population = dataset_population.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")
paises_unemployment = dataset_unemployment.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")
# paises_pib = dataset_pib.query("pais == 'Paraguay' or pais == 'Russian Federation' or pais == 'United States' or pais == 'United Kingdom' or pais == 'China' or pais == 'Netherlands' or pais == 'Spain'")
# paises_inflation = dataset_inflation.query("pais == 'Paraguay' or pais == 'Russian Federation' or pais == 'United States' or pais == 'United Kingdom' or pais == 'China' or pais == 'Netherlands' or pais == 'Spain'")
# paises_trade = dataset_trade.query("pais == 'Paraguay' or pais == 'Russian Federation' or pais == 'United States' or pais == 'United Kingdom' or pais == 'China' or pais == 'Netherlands' or pais == 'Spain'")
# paises_population = dataset_population.query("pais == 'Paraguay' or pais == 'Russian Federation' or pais == 'United States' or pais == 'United Kingdom' or pais == 'China' or pais == 'Netherlands' or pais == 'Spain'")
# paises_unemployment = dataset_unemployment.query("pais == 'Paraguay' or pais == 'Russian Federation' or pais == 'United States' or pais == 'United Kingdom' or pais == 'China' or pais == 'Netherlands' or pais == 'Spain'")

## Comparando

In [ ]:
# paises_pib
# paises_inflation
# paises_trade
# paises_population
paises_unemployment

In [ ]:
def plot_regressao_estimada(data, title, data_type):
    num_paises = math.ceil(len(data)/2)
    pib = make_subplots(rows=num_paises, cols=2, start_cell="bottom-left")
    pib.update_layout(
        title=title,
        width=1300,
        height=800,
        showlegend=False,
        colorway=['blue', 'green', 'orange', 'purple', 'brown', 'black', 'gray']
    )
    for i, pais in enumerate(data.pais):
        row = num_paises - (i // 2)
        col = (i % 2) + 1
        trace = go.Scatter(x=data.columns[1:], y=data.iloc[i, 1:], name=pais)
        x = np.array(data.columns[1:], dtype=data_type)
        y = np.array(data.iloc[i, 1:], dtype=data_type)
        coef = np.polyfit(x, y, 1)
        line = coef[1] + coef[0] * x
        reg = go.Scatter(x=x, y=line, mode='lines', name='Regressão', line=dict(color='red'))
        pib.add_trace(reg, row=row, col=col)
        pib.add_trace(trace, row=row, col=col)
        pib.update_xaxes(
            title_text=pais,
            row=row,
            col=col,
            title_standoff=10,
            tickfont=dict(size=8),
            title_font=dict(size=10),
            tickvals=data.columns[1:]
        )
    pib.show()

In [ ]:
def plot_regressao1(data, title, data_type):
    # num_paises = math.ceil(len(data)/2)
    #pib = make_subplots(rows=num_paises, cols=2, start_cell="bottom-left")
    pib=go.Figure()

    pib.update_layout(
        title=title,
        width=600,
        height=300,
        showlegend=False,
        colorway=['blue', 'green', 'orange', 'purple', 'brown', 'black', 'gray']
    )
    for i, pais in enumerate(data.pais):
        # row = num_paises - (i // 2)
        # col = (i % 2) + 1
        trace = go.Scatter(x=data.columns[1:], y=data.iloc[i, 1:], name=pais)
        x = np.array(data.columns[1:], dtype=data_type)
        y = np.array(data.iloc[i, 1:], dtype=data_type)
        coef = np.polyfit(x, y, 1)
        line = coef[1] + coef[0] * x
        reg = go.Scatter(x=x, y=line, mode='lines', name='Regressão', line=dict(color='red'))
        pib.add_trace(reg)
        pib.add_trace(trace)
        pib.update_xaxes(
            title_text=pais,
            # row=row,
            # col=col,
            title_standoff=10,
            tickfont=dict(size=8),
            title_font=dict(size=10),
            tickvals=data.columns[1:]
        )
    pib.show()

In [ ]:
plot_regressao1(paises_exportacao.query("pais=='Haiti'"), 'Regressão estimada dos valores de exportação para o Haiti', int)

In [ ]:
plot_regressao_estimada(paises_exportacao, 'Comparação da regressão estimada dos valores de exportação para os principais países', int)

In [ ]:
plot_regressao_estimada(paises_importacao, 'Comparação da regressão estimada dos valores de importação para os principais países', int)

In [ ]:
plot_regressao_estimada(paises_pib, 'Comparação da regressão estimada do PIB dos países responsáveis por 80% da exportação', float)

In [ ]:
plot_regressao_estimada(paises_inflation, 'Comparação da regressão estimada da inflação dos países responsáveis por 80% da exportação', float)

In [ ]:
plot_regressao_estimada(paises_trade, 'Comparação da regressão estimada do comércio internacional do países responsáveis por 80% da exportação', float)

In [ ]:
plot_regressao_estimada(paises_population, 'Comparação da regressão estimada da população dos países responsáveis por 80% da exportação', int)

In [ ]:
plot_regressao_estimada(paises_unemployment, 'Comparação da regressão estimada do desemprego dos países responsáveis por 80% da exportação', float)

## Comparação dos países mais representativos

In [ ]:
def plot_regressao_estimada_comparacao(data, title, data_type):
    data = data.query("pais == 'Paraguai' or pais == 'Rússia'")
    num_paises = len(data)
    pib = make_subplots(rows=num_paises, cols=1)
    pib.update_layout(
        title=title,
        width=1300,
        height=600,
        showlegend=False
    )
    for i, pais in enumerate(data.pais):
        row = i + 1
        trace = go.Scatter(x=data.columns[1:], y=data.iloc[i, 1:], name=pais)
        x = np.array(data.columns[1:], dtype=float)
        y = np.array(data.iloc[i, 1:], dtype=float)
        coef = np.polyfit(x, y, 1)
        line = coef[1] + coef[0] * x
        reg = go.Scatter(x=x, y=line, mode='lines', name='Regressão', line=dict(color='red'))
        pib.add_trace(reg, row=row, col=1)
        pib.add_trace(trace, row=row, col=1)
        pib.update_xaxes(title_text=pais, row=row, col=1, title_standoff=10, tickfont=dict(size=8), title_font=dict(size=10))
    pib.show()

In [ ]:
plot_regressao_estimada_comparacao(paises_exportacao, 'Comparação da regressão estimada dos valores de exportação para Paraguai e Rússia', int)

In [ ]:
plot_regressao_estimada_comparacao(paises_pib, 'Comparação do PIB dos países Paraguai e Rússia', float)

In [ ]:
plot_regressao_estimada_comparacao(paises_inflation, 'Comparação da inflação dos países Paraguai e Rússia', float)

In [ ]:
plot_regressao_estimada_comparacao(paises_trade, 'Comparação do índice de comércio internacional dos países Paraguai e Rússia', float)

In [ ]:
plot_regressao_estimada_comparacao(paises_population, 'Comparação da população dos países Paraguai e Rússia', int)

In [ ]:
plot_regressao_estimada_comparacao(paises_unemployment, 'Comparação do desemprego dos países Paraguai e Rússia', float)

In [ ]:
dataset_2015 = pd.read_csv('Data/2015.csv', sep=',')
dataset_2015 = dataset_2015[['Country','Happiness Score']]
dataset_2015.rename(columns={"Country": "pais", "Happiness Score": "2015"}, inplace=True)
dataset_2016 = pd.read_csv('Data/2016.csv', sep=',')
dataset_2016 = dataset_2016[['Country','Happiness Score']]
dataset_2016.rename(columns={"Country": "pais", "Happiness Score": "2016"}, inplace=True)
dataset_2017 = pd.read_csv('Data/2017.csv', sep=',')
dataset_2017 = dataset_2017[['Country','Happiness.Score']]
dataset_2017.rename(columns={"Country": "pais", "Happiness.Score": "2017"}, inplace=True)
dataset_2018 = pd.read_csv('Data/2018.csv', sep=',')
dataset_2018 = dataset_2018[['Country or region','Score']]
dataset_2018.rename(columns={"Country or region": "pais", "Score": "2018"}, inplace=True)
dataset_2019 = pd.read_csv('Data/2019.csv', sep=',')
dataset_2019 = dataset_2019[['Country or region','Score']]
dataset_2019.rename(columns={"Country or region": "pais", "Score": "2019"}, inplace=True)
dataset_2019.head()

In [ ]:
dataset_whr = pd.merge(dataset_2015, dataset_2016, on='pais')
dataset_whr = pd.merge(dataset_whr, dataset_2017, on='pais')
dataset_whr = pd.merge(dataset_whr, dataset_2018, on='pais')
dataset_whr = pd.merge(dataset_whr, dataset_2019, on='pais')
nomes = {
    "Switzerland": "Suíça",
    "Iceland": "Islândia",
    "Denmark": "Dinamarca",
    "Norway": "Noruega",
    "Canada": "Canadá",
    "Finland": "Finlândia",
    "Netherlands": "Países Baixos",
    "Sweden": "Suécia",
    "New Zealand": "Nova Zelândia",
    "Australia": "Austrália",
    "Israel": "Israel",
    "Costa Rica": "Costa Rica",
    "Austria": "Áustria",
    "Mexico": "México",
    "United States": "Estados Unidos",
    "Brazil": "Brasil",
    "Luxembourg": "Luxemburgo",
    "Ireland": "Irlanda",
    "Belgium": "Bélgica",
    "United Arab Emirates": "Emirados Árabes Unidos",
    "United Kingdom": "Reino Unido",
    "Venezuela": "Venezuela",
    "Singapore": "Singapura",
    "Panama": "Panamá",
    "Germany": "Alemanha",
    "Chile": "Chile",
    "Qatar": "Catar",
    "France": "França",
    "Argentina": "Argentina",
    "Czech Republic": "República Tcheca",
    "Uruguay": "Uruguai",
    "Colombia": "Colômbia",
    "Thailand": "Tailândia",
    "Saudi Arabia": "Arábia Saudita",
    "Spain": "Espanha",
    "Malta": "Malta",
    "Kuwait": "Kuwait",
    "El Salvador": "El Salvador",
    "Guatemala": "Guatemala",
    "Uzbekistan": "Uzbequistão",
    "Slovakia": "Eslováquia",
    "Japan": "Japão",
    "South Korea": "Coreia do Sul",
    "Ecuador": "Equador",
    "Bahrain": "Bahrein",
    "Italy": "Itália",
    "Bolivia": "Bolívia",
    "Moldova": "Moldávia",
    "Paraguay": "Paraguai",
    "Kazakhstan": "Cazaquistão",
    "Slovenia": "Eslovênia",
    "Lithuania": "Lituânia",
    "Nicaragua": "Nicarágua",
    "Peru": "Peru",
    "Belarus": "Bielorrússia",
    "Poland": "Polônia",
    "Malaysia": "Malásia",
    "Croatia": "Croácia",
    "Libya": "Líbia",
    "Russia": "Rússia",
    "Jamaica": "Jamaica",
    "Cyprus": "Chipre",
    "Algeria": "Argélia",
    "Kosovo": "Kosovo",
    "Turkmenistan": "Turcomenistão",
    "Mauritius": "Maurício",
    "Estonia": "Estônia",
    "Indonesia": "Indonésia",
    "Vietnam": "Vietnã",
    "Turkey": "Turquia",
    "Kyrgyzstan": "Quirguistão",
    "Nigeria": "Nigéria",
    "Bhutan": "Butão",
    "Azerbaijan": "Azerbaijão",
    "Pakistan": "Paquistão",
    "Jordan": "Jordânia",
    "Montenegro": "Montenegro",
    "China": "China",
    "Zambia": "Zâmbia",
    "Romania": "Romênia",
    "Serbia": "Sérvia",
    "Portugal": "Portugal",
    "Latvia": "Letônia",
    "Philippines": "Filipinas",
    "Morocco": "Marrocos",
    "Albania": "Albânia",
    "Bosnia and Herzegovina": "Bósnia e Herzegovina",
    "Dominican Republic": "República Dominicana",
    "Mongolia": "Mongólia",
    "Greece": "Grécia",
    "Lebanon": "Líbano",
    "Hungary": "Hungria",
    "Honduras": "Honduras",
    "Tajikistan": "Tajiquistão",
    "Tunisia": "Tunísia",
    "Palestinian Territories": "Palestina",
    "Bangladesh": "Bangladesh",
    "Iran": "Irã",
    "Ukraine": "Ucrânia",
    "Iraq": "Iraque",
    "South Africa": "África do Sul",
    "Ghana": "Gana",
    "Zimbabwe": "Zimbábue",
    "Liberia": "Libéria",
    "India": "Índia",
    "Haiti": "Haiti",
    "Congo (Kinshasa)": "Dem Congo",
    "Nepal": "Nepal",
    "Ethiopia": "Etiópia",
    "Sierra Leone": "Serra Leoa",
    "Mauritania": "Mauritânia",
    "Kenya": "Quênia",
    "Armenia": "Armênia",
    "Botswana": "Botsuana",
    "Myanmar": "Mianmar (Birmânia)",
    "Georgia": "Geórgia",
    "Malawi": "Malawi",
    "Sri Lanka": "Sri Lanka",
    "Cameroon": "Camarões",
    "Bulgaria": "Bulgária",
    "Egypt": "Egito",
    "Yemen": "Iêmen",
    "Mali": "Mali",
    "Congo (Brazzaville)": "Congo",
    "Uganda": "Uganda",
    "Senegal": "Senegal",
    "Gabon": "Gabão",
    "Niger": "Níger",
    "Cambodia": "Camboja",
    "Tanzania": "Tanzânia",
    "Madagascar": "Madagascar",
    "Chad": "Chade",
    "Guinea": "Guiné",
    "Ivory Coast": "Costa do Marfim",
    "Burkina Faso": "Burkina Faso",
    "Afghanistan": "Afeganistão",
    "Rwanda": "Ruanda",
    "Benin": "Benin",
    "Syria": "Síria",
    "Burundi": "Burundi",
    "Togo": "Togo"
}

dataset_whr["pais"] = dataset_whr["pais"].map(nomes)
dataset_whr.head(1)

### Salvar .CSV para uso no Streamlit

In [ ]:
dataset_whr.to_csv('wht.csv', sep=',', index=False)

In [ ]:
dataset_whr = dataset_whr.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")

In [ ]:
plot_regressao_estimada(dataset_whr, 'Comparação da regressão estimada dos valores de felicidade para os principais países', int)

In [ ]:
dataset_consumo = pd.read_csv('Data/consumption.csv', sep=',')
dataset_consumo = dataset_consumo[['Location', 'Period', 'FactValueNumeric']]
dataset_2020 = dataset_consumo.query("Period == 2020").reset_index()
dataset_2020.Period = dataset_2020.FactValueNumeric
dataset_2020.rename(columns={'Location': 'pais', 'Period': '2020'}, inplace=True)
dataset_2020 = dataset_2020[['pais', '2020']]
dataset_2025 = dataset_consumo.query("Period == 2025").reset_index()
dataset_2025.Period = dataset_2025.FactValueNumeric
dataset_2025 = dataset_2025.rename(columns={'Location': 'pais', 'Period': '2025',})
dataset_2025 = dataset_2025[['pais', '2025']]
dataset_consumo = dataset_2020.merge(dataset_2025, on='pais')
dataset_consumo = dataset_consumo[dataset_consumo['2025'] != 0.0]
nomes = {
    'Afghanistan': 'Afeganistão',
    'Albania': 'Albânia',
    'Algeria': 'Argélia',
    'Andorra': 'Andorra',
    'Angola': 'Angola',
    'Antigua and Barbuda': 'Antígua e Barbuda',
    'Argentina': 'Argentina',
    'Armenia': 'Armênia',
    'Australia': 'Austrália',
    'Austria': 'Áustria',
    'Azerbaijan': 'Azerbaijão',
    'Bahamas': 'Bahamas',
    'Bahrain': 'Bahrein',
    'Bangladesh': 'Bangladesh',
    'Barbados': 'Barbados',
    'Belarus': 'Bielorrússia',
    'Belgium': 'Bélgica',
    'Belize': 'Belize',
    'Benin': 'Benin',
    'Bhutan': 'Butão',
    'Bolivia (Plurinational State of)': 'Bolívia',
    'Bosnia and Herzegovina': 'Bósnia e Herzegovina',
    'Botswana': 'Botsuana',
    'Brazil': 'Brasil',
    'Brunei Darussalam': 'Brunei',
    'Bulgaria': 'Bulgária',
    'Burkina Faso': 'Burquina Faso',
    'Burundi': 'Burundi',
    'Cambodia': 'Camboja',
    'Cameroon': 'Camarões',
    'Canada': 'Canadá',
    'Cabo Verde': 'Cabo Verde',
    'Central African Republic': 'República Centro-Africana',
    'Chad': 'Chade',
    'Chile': 'Chile',
    'China': 'China',
    'Colombia': 'Colômbia',
    'Comoros': 'Comores',
    'Congo': 'Congo',
    'Cook Islands': 'Ilhas Cook',
    'Costa Rica': 'Costa Rica',
    "Côte d’Ivoire": 'Costa do Marfim',
    'Croatia': 'Croácia',
    'Cuba': 'Cuba',
    'Cyprus': 'Chipre',
    'Czechia': 'República Tcheca',
    'Democratic People\'s Republic of Korea': 'Coreia do Norte',
    'Democratic Republic of the Congo': 'Dem Congo',
    'Denmark': 'Dinamarca',
    'Djibouti': 'Djibuti',
    'Dominica': 'Dominica',
    'Dominican Republic': 'República Dominicana',
    'Ecuador': 'Equador',
    'Egypt': 'Egito',
    'El Salvador': 'El Salvador',
    'Equatorial Guinea': 'Guiné Equatorial',
    'Eritrea': 'Eritreia',
    'Estonia': 'Estônia',
    'Eswatini': 'Eswatini',
    'Ethiopia': 'Etiópia',
    'Fiji': 'Fiji',
    'Finland': 'Finlândia',
    'France': 'França',
    'Gabon': 'Gabão',
    'Gambia': 'Gâmbia',
    'Georgia': 'Geórgia',
    'Germany': 'Alemanha',
    'Ghana': 'Gana',
    'Greece': 'Grécia',
    'Grenada': 'Granada',
    'Guatemala': 'Guatemala',
    'Guinea': 'Guiné',
    'Guinea-Bissau': 'Guiné-Bissau',
    'Guyana': 'Guiana',
    'Haiti': 'Haiti',
    'Honduras': 'Honduras',
    'Hungary': 'Hungria',
    'Iceland': 'Islândia',
    'India': 'Índia',
    'Indonesia': 'Indonésia',
    'Iran (Islamic Republic of)': 'Irã',
    'Iraq': 'Iraque',
    'Ireland': 'Irlanda',
    'Israel': 'Israel',
    'Italy': 'Itália',
    'Jamaica': 'Jamaica',
    'Japan': 'Japão',
    'Jordan': 'Jordânia',
    'Kazakhstan': 'Cazaquistão',
    'Kenya': 'Quênia',
    'Kiribati': 'Kiribati',
    'Kuwait': 'Kuwait',
    'Kyrgyzstan': 'Quirguistão',
    "Lao People's Democratic Republic": 'Laos',
    'Latvia': 'Letônia',
    'Lebanon': 'Líbano',
    'Lesotho': 'Lesoto',
    'Liberia': 'Libéria',
    'Libya': 'Líbia',
    'Liechtenstein': 'Liechtenstein',
    'Lithuania': 'Lituânia',
    'Luxembourg': 'Luxemburgo',
    'Madagascar': 'Madagascar',
    'Malawi': 'Malawi',
    'Malaysia': 'Malásia',
    'Maldives': 'Maldivas',
    'Mali': 'Mali',
    'Malta': 'Malta',
    'Marshall Islands': 'Ilhas Marshall',
    'Mauritania': 'Mauritânia',
    'Mauritius': 'Maurício',
    'Mexico': 'México',
    'Micronesia (Federated States of)': 'Micronésia',
    'Monaco': 'Mônaco',
    'Mongolia': 'Mongólia',
    'Montenegro': 'Montenegro',
    'Morocco': 'Marrocos',
    'Mozambique': 'Moçambique',
    'Myanmar': 'Myanmar',
    'Namibia': 'Namíbia',
    'Nauru': 'Nauru',
    'Nepal': 'Nepal',
    'Netherlands': 'Países Baixos',
    'New Zealand': 'Nova Zelândia',
    'Nicaragua': 'Nicarágua',
    'Niger': 'Níger',
    'Nigeria': 'Nigéria',
    'Niue': 'Niue',
    'North Macedonia': 'Macedônia do Norte',
    'Norway': 'Noruega',
    'Oman': 'Omã',
    'Pakistan': 'Paquistão',
    'Palau': 'Palau',
    'Panama': 'Panamá',
    'Papua New Guinea': 'Papua-Nova Guiné',
    'Paraguay': 'Paraguai',
    'Peru': 'Peru',
    'Philippines': 'Filipinas',
    'Poland': 'Polônia',
    'Portugal': 'Portugal',
    'Qatar': 'Catar',
    'Republic of Korea': 'Coreia do Sul',
    'Republic of Moldova': 'Moldávia',
    'Romania': 'Romênia',
    'Russian Federation': 'Rússia',
    'Rwanda': 'Ruanda',
    'Saint Kitts and Nevis': 'São Cristóvão e Nevis',
    'Saint Lucia': 'Santa Lúcia',
    'Saint Vincent and the Grenadines': 'São Vicente e Granadinas',
    'Samoa': 'Samoa',
    'San Marino': 'San Marino',
    'Sao Tome and Principe': 'São Tomé e Príncipe',
    'Saudi Arabia': 'Arábia Saudita',
    'Senegal': 'Senegal',
    'Serbia': 'Sérvia',
    'Seychelles': 'Seychelles',
    'Sierra Leone': 'Serra Leoa',
    'Singapore': 'Singapura',
    'Slovakia': 'Eslováquia',
    'Slovenia': 'Eslovênia',
    'Solomon Islands': 'Ilhas Salomão',
    'Somalia': 'Somália',
    'South Africa': 'África do Sul',
    'South Sudan': 'Sudão do Sul',
    'Spain': 'Espanha',
    'Sri Lanka': 'Sri Lanka',
    'Sudan': 'Sudão',
    'Suriname': 'Suriname',
    'Sweden': 'Suécia',
    'Switzerland': 'Suíça',
    'Syrian Arab Republic': 'Síria',
    'Tajikistan': 'Tajiquistão',
    'Thailand': 'Tailândia',
    'Timor-Leste': 'Timor-Leste',
    'Togo': 'Togo',
    'Tonga': 'Tonga',
    'Trinidad and Tobago': 'Trinidad e Tobago',
    'Tunisia': 'Tunísia',
    'Turkey': 'Turquia',
    'Turkmenistan': 'Turcomenistão',
    'Tuvalu': 'Tuvalu',
    'Uganda': 'Uganda',
    'Ukraine': 'Ucrânia',
    'United Arab Emirates': 'Emirados Árabes Unidos',
    'United Kingdom of Great Britain and Northern Ireland': 'Reino Unido',
    'United Republic of Tanzania': 'Tanzânia',
    'United States of America': 'Estados Unidos',
    'Uruguay': 'Uruguai',
    'Uzbekistan': 'Uzbequistão',
    'Vanuatu': 'Vanuatu',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnã',
    'Yemen': 'Iêmen',
    'Zambia': 'Zâmbia',
    'Zimbabwe': 'Zimbábue'
}

dataset_consumo["pais"] = dataset_consumo["pais"].map(nomes)
dataset_consumo["diferenca"] = dataset_consumo["2025"] - dataset_consumo["2020"]
dataset_consumo = dataset_consumo.dropna()
dataset_consumo.sort_values('2025', ascending=False, inplace=True)
dataset_consumo

### Salvar .CSV para uso no Streamlit

In [ ]:
dataset_consumo.to_csv('Consumo.csv', sep=',', index=False)

In [ ]:
consumo_paises = dataset_consumo.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")
# dados_paises = dataset_consumo[dataset_consumo["pais"].isin(paises)]
consumo = go.Figure()
consumo.add_trace(go.Bar(x=consumo_paises.pais, y=consumo_paises["2020"], name="2020", marker=dict(color="lightblue")))
consumo.add_trace(go.Bar(x=consumo_paises.pais, y=consumo_paises["2025"], name="2025", marker=dict(color="lightgreen")))
for i, pais in enumerate(consumo_paises.pais):
    diff = round(consumo_paises["diferenca"].iloc[i], 2)
    consumo.add_annotation(x=pais, y=max(consumo_paises["2020"].iloc[i], consumo_paises["2025"].iloc[i]),
                            text=f"Diferença: {diff}",
                            showarrow=False,
                            font=dict(color="black"),
                            yshift=10
                       )
consumo.update_layout(title="Projeção do consumo de álcool em 2020 e 2025",
                    xaxis_title="Países",
                    yaxis_title="Consumo",
                    barmode="group"
                )
consumo.show()

In [ ]:
dataset_consumo_vinho = pd.read_excel('Data/data.xlsx')
dataset_consumo_vinho = dataset_consumo_vinho[['Region/Country', 'Year', 'Quantity']]
dataset_consumo_vinho.rename(columns={'Region/Country': 'pais', 'Year': 'ano', 'Quantity': 'quantidade'}, inplace=True)
dataset_consumo_vinho['ano'] = dataset_consumo_vinho['ano'].astype('Int64')
dataset_consumo_vinho.head(20)

In [ ]:
dataset_consumo_vinho = dataset_consumo_vinho.pivot_table(index='pais', columns='ano', values='quantidade')
dataset_consumo_vinho = dataset_consumo_vinho.reset_index()
dataset_consumo_vinho.index.name = None
dataset_consumo_vinho.head()

In [ ]:
nomes = {
    'Afghanistan': 'Afeganistão',
    'Albania': 'Albânia',
    'Algeria': 'Argélia',
    'American Samoa': 'Samoa Americana',
    'Angola': 'Angola',
    'Antigua and Barbuda': 'Antígua e Barbuda',
    'Argentina': 'Argentina',
    'Armenia': 'Armênia',
    'Australia': 'Austrália',
    'Austria': 'Áustria',
    'Azerbaijan': 'Azerbaijão',
    'Bahamas': 'Bahamas',
    'Bahrain': 'Bahrein',
    'Bangladesh': 'Bangladesh',
    'Barbados': 'Barbados',
    'Belarus': 'Bielorrússia',
    'Belgium': 'Bélgica',
    'Belize': 'Belize',
    'Benin': 'Benin',
    'Bermuda': 'Bermudas',
    'Bolivia': 'Bolívia',
    'Bosnia and Herzegovina': 'Bósnia e Herzegovina',
    'Botswana': 'Botsuana',
    'Brazil': 'Brasil',
    'Brunei Darussalam': 'Brunei',
    'Bulgaria': 'Bulgária',
    'Burkina Faso': 'Burkina Faso',
    'Burundi': 'Burundi',
    'Cabo Verde': 'Cabo Verde',
    'Cambodia': 'Camboja',
    'Cameroon': 'Camarões',
    'Canada': 'Canadá',
    'Central African Republic': 'República Centro-Africana',
    'Chad': 'Chade',
    'Chile': 'Chile',
    'China': 'China',
    'Colombia': 'Colômbia',
    'Comoros': 'Comores',
    'Congo': 'Congo',
    'Congo (the Democratic Republic of the)': 'Dem Congo',
    'Cook Islands': 'Ilhas Cook',
    'Costa Rica': 'Costa Rica',
    'Croatia': 'Croácia',
    'Cuba': 'Cuba',
    'Cyprus': 'Chipre',
    'Czech Republic': 'República Tcheca',
    "Côte d'Ivoire": 'Costa do Marfim',
    'Denmark': 'Dinamarca',
    'Djibouti': 'Djibuti',
    'Dominica': 'Dominica',
    'Dominican Republic': 'República Dominicana',
    'Ecuador': 'Equador',
    'Egypt': 'Egito',
    'El Salvador': 'El Salvador',
    'Estonia': 'Estônia',
    'Eswatini': 'Suazilândia',
    'Ethiopia': 'Etiópia',
    'Faroe Islands': 'Ilhas Faroe',
    'Fiji': 'Fiji',
    'Finland': 'Finlândia',
    'France': 'França',
    'French Guiana': 'Guiana Francesa',
    'French Polynesia': 'Polinésia Francesa',
    'Gabon': 'Gabão',
    'Gambia': 'Gâmbia',
    'Georgia': 'Geórgia',
    'Germany': 'Alemanha',
    'Ghana': 'Gana',
    'Global': 'Global',
    'Greece': 'Grécia',
    'Greenland': 'Groenlândia',
    'Grenada': 'Granada',
    'Guadeloupe': 'Guadalupe',
    'Guam': 'Guam',
    'Guinea': 'Guiné',
    'Guinea-Bissau': 'Guiné-Bissau',
    'Guyana': 'Guiana',
    'Haiti': 'Haiti',
    'Honduras': 'Honduras',
    'Hong Kong': 'Hong Kong',
    'Hungary': 'Hungria',
    'Iceland': 'Islândia',
    'India': 'Índia',
    'Indonesia': 'Indonésia',
    'Iran': 'Irã',
    'Iraq': 'Iraque',
    'Ireland': 'Irlanda',
    'Israel': 'Israel',
    'Italy': 'Itália',
    'Jamaica': 'Jamaica',
    'Japan': 'Japão',
    'Jordan': 'Jordânia',
    'Kazakhstan': 'Cazaquistão',
    'Kenya': 'Quênia',
    'Kiribati': 'Kiribati',
    'Korea (the Republic of)': 'Coreia do Sul',
    'Kyrgyzstan': 'Quirguistão',
    "Lao People's Democratic Republic": 'Laos',
    'Latvia': 'Letônia',
    'Lebanon': 'Líbano',
    'Lesotho': 'Lesoto',
    'Liberia': 'Libéria',
    'Libya': 'Líbia',
    'Lithuania': 'Lituânia',
    'Luxemburg': 'Luxemburgo',
    'Macao': 'Macau',
    'Madagascar': 'Madagascar',
    'Malawi': 'Malawi',
    'Malaysia': 'Malásia',
    'Maldives': 'Maldivas',
    'Mali': 'Mali',
    'Malta': 'Malta',
    'Martinique': 'Martinica',
    'Mauritania': 'Mauritânia',
    'Mauritius': 'Maurício',
    'Mexico': 'México',
    'Moldova': 'Moldávia',
    'Mongolia': 'Mongólia',
    'Montenegro': 'Montenegro',
    'Montserrat': 'Montserrat',
    'Morocco': 'Marrocos',
    'Mozambique': 'Moçambique',
    'Myanmar': 'Mianmar',
    'Namibia': 'Namíbia',
    'Nauru': 'Nauru',
    'Nepal': 'Nepal',
    'Netherlands': 'Países Baixos',
    'Netherlands Antilles': 'Antilhas Holandesas',
    'New Caledonia': 'Nova Caledônia',
    'New Zealand': 'Nova Zelândia',
    'Nicaragua': 'Nicarágua',
    'Niger': 'Níger',
    'Nigeria': 'Nigéria',
    'Niue': 'Niue',
    'Norway': 'Noruega',
    'Pakistan': 'Paquistão',
    'Panama': 'Panamá',
    'Papua New Guinea': 'Papua-Nova Guiné',
    'Paraguay': 'Paraguai',
    'Peru': 'Peru',
    'Philippines': 'Filipinas',
    'Poland': 'Polônia',
    'Portugal': 'Portugal',
    'Republic of North Macedonia': 'Macedônia do Norte',
    'Republic of Türkiye': 'Turquia',
    'Reunion island': 'Reunião',
    'Romania': 'Romênia',
    'Russia': 'Rússia',
    'Rwanda': 'Ruanda',
    'Saint Helena': 'Santa Helena',
    'Saint Kitts and Nevis': 'São Cristóvão e Nevis',
    'Saint Lucia': 'Santa Lúcia',
    'Saint Pierre And Miquelon': 'Saint-Pierre e Miquelon',
    'Saint Vincent and the Grenadines': 'São Vicente e Granadinas',
    'Samoa': 'Samoa',
    'Sao Tome and Principe': 'São Tomé e Príncipe',
    'Saudi Arabia': 'Arábia Saudita',
    'Senegal': 'Senegal',
    'Serbia': 'Sérvia',
    'Seychelles': 'Seicheles',
    'Sierra Leone': 'Serra Leoa',
    'Singapore': 'Singapura',
    'Slovakia': 'Eslováquia',
    'Slovenia': 'Eslovênia',
    'Solomon Islands': 'Ilhas Salomão',
    'Somalia': 'Somália',
    'South Africa': 'África do Sul',
    'Spain': 'Espanha',
    'Sri Lanka': 'Sri Lanka',
    'State of Palestine': 'Palestina',
    'Sudan': 'Sudão',
    'Suriname': 'Suriname',
    'Sweden': 'Suécia',
    'Switzerland': 'Suíça',
    'Syrian Arab Republic': 'Síria',
    'Taiwan': 'Taiwan',
    'Tajikistan': 'Tajiquistão',
    'Tanzania, the United Republic of': 'Tanzânia',
    'Thailand': 'Tailândia',
    'Timor-Leste': 'Timor-Leste',
    'Togo': 'Togo',
    'Tonga': 'Tonga',
    'Trinidad and Tobago': 'Trinidad e Tobago',
    'Tunisia': 'Tunísia',
    'Turkmenistan': 'Turcomenistão',
    'Tuvalu': 'Tuvalu',
    'Uganda': 'Uganda',
    'Ukraine': 'Ucrânia',
    'United Kingdom': 'Reino Unido',
    'United States of America': 'Estados Unidos',
    'Uruguay': 'Uruguai',
    'Uzbekistan': 'Uzbequistão',
    'Vanuatu': 'Vanuatu',
    'Venezuela': 'Venezuela',
    'Vietnam': 'Vietnã',
    'Yemen': 'Iêmen',
    'Zambia': 'Zâmbia',
    'Zimbabwe': 'Zimbábue'
}

dataset_consumo_vinho["pais"] = dataset_consumo_vinho["pais"].map(nomes)

In [ ]:
dataset_consumo_vinho.head()

### Salvar .CSV para uso no Streamlit

In [ ]:
dataset_consumo_vinho.to_csv('ConsumoVinho.csv', sep=',', index=False)

In [ ]:
dataset_consumo_vinho = dataset_consumo_vinho.query("pais == 'Paraguai' or pais == 'Rússia' or pais == 'Estados Unidos' or pais == 'Reino Unido' or pais == 'China' or pais == 'Países Baixos' or pais == 'Espanha'")

In [ ]:
plot_regressao_estimada(dataset_consumo_vinho, 'Comparação da regressão estimada do consumo de vinho para os principais países', float)